# Additional features and functionalities of flowerMD

## Overview:
In this tutorial, we present additional features and functionalities within flowerMD, empowering users to customize their simulations according to their specific requirements. We will cover the following topics for flowerMD  modules:

**1) Molecule**

- Custom Molecule definition
- Polymer and CoPolymer Builders
  
**2) ForceField**

- Custom ForceFields (XML-based)
- Custom HOOMD Forces

**3) System**
   
- Custom assembly algorithms
- Mixture of molecules
- Reference values (auto scaling)

**4) Simulation**
- Simulation methods
- Resuming a simulation



## 1. Molecule

### Custom Molecule definition

You can define custom molecules in a couple of different ways:

1. Define molecules using their [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) string representation.
2. Utilize molecule files in formats such as `.mol` or `.sdf` to specify your custom molecules.
3. Create molecules from an [`mBuild`](https://mbuild.mosdef.org/en/stable/) compound or a [`GMSO`](https://gmso.mosdef.org/en/stable/) topology.
4. Customize molecules by creating a subclass of the `Molecule` class.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#### Option 1: Using the SMILES string of the molecule

In [ ]:
from flowermd.base import Molecule

benzoic_acid_mol = Molecule(num_mols=20, smiles="c1cc(C(O)=O)ccc1")

We can use the `mBuild` visualization function to visualize one of the 20 benzoic acid molecules.

In [ ]:
benzoic_acid_mol.molecules[0].visualize()

#### Option 2: Initialize molecule from a file

In [ ]:
phenol_mol = Molecule(num_mols=20, file="../flowermd/assets/molecule_files/IPH.mol2")

In [ ]:
phenol_mol.molecules[0].visualize()

#### Option 3: Start molecule from an [`mBuild`](https://mbuild.mosdef.org/en/stable/) compound or a [`GMSO`](https://gmso.mosdef.org/en/stable/) topology

In [ ]:
import mbuild as mb

mb_compound = mb.load("c1ccccc1", smiles=True)

gmso_top = mb_compound.to_gmso()

benzene_mol = Molecule(num_mols=20, compound=mb_compound)
benzene_mol = Molecule(num_mols=20, compound=gmso_top)


#### Option 4: Define a subclass of the `Molecule` class

Users can extend the molecule class functionalities by creating subclasses of the `Molecule` base class. This can be achieved by overriding the `_generate` method, which is responsible for generating all the molecules in form of [mbuild compounds](https://mbuild.mosdef.org/en/stable/topic_guides/data_structures.html#compound) and appending them to the `self._molecules` list.

flowerMD implements two subclasses of the `Molecule` class: `Polymer` and `CoPolymer`. Please see [flowerMD's Molecule documentation](https://flowermd.readthedocs.io/en/latest/molecule.html) for more details.

## 2. Forcefields

flowerMD offers a range of predefined forcefield classes that can be used to parameterize a system. Checkout  [flowerMD's library documentation](https://flowermd.readthedocs.io/en/latest/forcefields.html) to see the list of available forcefields.

Users can also define their own custom forcefields with the following two options:

### Option 1: XML-based custom forcefields

Given the path to the XML file of a desired forcefield, users can employ the `FF_from_file` class available in `flowermd.library` to instantiate a forcefield object.

*Note*: The XML file should be in the format of [foyer](https://foyer.mosdef.org/en/stable/getting_started/quickstart.html) forcefield files.

In [ ]:
from flowermd.library import FF_from_file

benzene_ff = FF_from_file(forcefield_files="../flowermd/assets/forcefields/benzene_opls.xml")

### Option 2: HOOMD Forces custom forcefields


Users have the flexibility to directly define their own list of HOOMD force objects tailored to specific simulation requirements rather than initiating it from an XML based forcefield. In such cases, users can specify the forces by creating a subclass of the `flowermd.base.BaseHOOMDForcefield` class. This can be achieved by developing `_create_forcefield` method, which is responsible for generating a list of `hoomd.md.force` objects. Below is an example of defining `BreadSpring` custom class that creates HOOMD force objects for a system of coarse-grained beads given the parameters for bonded and non-bonded interactions.


In [ ]:
from flowermd.base import BaseHOOMDForcefield
class BeadSpring(BaseHOOMDForcefield):
     def __init__(self, r_cut, beads, bonds=None, angles=None, dihedrals=None,exclusions=["bond", "1-3"]):
        # ...
        hoomd_forces = self._create_forcefield()
        super(BeadSpring, self).__init__(hoomd_forces)

    def _create_forcefield(self):
        hoomd_forces = []
        # generate hoomd forces for pair interactions, bonds, angles, dihedrals, etc.
        # ...
        return hoomd_forces



Please also see [flowerMD's Forcefield documentation](https://flowermd.readthedocs.io/en/latest/forcefields.html) for more examples.


## 3. System

### Custom system assembly algorithms

The `System` class provides two distinct methods for filling the simulation box: `Pack` and `Lattice`.

`Pack` fills the simulation box with molecules in a way that closely packs them, typically in a disordered, random fashion.

`Lattice`, on the other hand, fills the simulation box using a well-defined lattice or grid structure. The `x` and `y` parameters specify the distance (in nm) between lattice points in x and y directions respectively. The `n` parameter determines the number of times to repear the unit cell in x and y directions.

In [ ]:
# example of defining a system using the Lattice method
from flowermd.base import Lattice
from flowermd.library import PPS, OPLS_AA_PPS

pps = PPS(num_mols=32, lengths=5)

lattice = Lattice(molecules=pps, density=0.8, x=1, y=1, n=4)

In [ ]:
lattice.system.visualize()

Note: Both `Pack` and `Lattice` classes are subclasses of the `System` class. The base `System` class is designed as an abstract class, meaning it's not intended to be directly instantiated. Check out `Pack` and `Lattice` [documentation](https://flowermd.readthedocs.io/en/latest/system.html) for more details .

If you desire to customize a molecule assembly algorithm that suits your specific requirements, you should create a subclass of the `System` class and override the abstract method `_build_system`. This method is responsible for organizing molecules from the `Molecule` class into a simulation box and returning the resulting mbuild compound.



### Mixture of molecules

flowerMD allows users to construct a system consisting of a mixture of molecule types, each potentially utilizing different force fields. If all the molecule types within the system share the same forcefield, then you only need to pass the forcefield once.

In [ ]:
from flowermd.base import Pack
from flowermd.library import OPLS_AA_DIMETHYLETHER
dimethylether_mol = Molecule(num_mols=20, smiles="COC")
pps_mol = PPS(num_mols=10, lengths=4)
multi_type_system = Pack( molecules=[dimethylether_mol, pps_mol], density=0.8)
multi_type_system.apply_forcefield(r_cut=2.5, force_field=[OPLS_AA_DIMETHYLETHER(), OPLS_AA_PPS()], auto_scale=True)

In [ ]:
multi_type_system.system.visualize()

### AutoScale & Reference Values

As mentioned in the previous tutorial, when the `auto_scale` parameter is set to `True` during system building, the parameters of the forcefields will be scaled automatically to range between 0 to 1. The scaling factor for these conversions can be retrieved from `system.reference_values`, which includes scaling factors for energy, length and mass values.

## 4. Simulation

### Simulation methods

Simulation methods/functionalities available in flowerMD `Simulation` class:


- `adjust_epsilon`: scales or shifts all epsilon values of LJ forces by a specified value. 
- `adjust_sigma`: scales or shifts all sigma values of LJ forces by a specified value.
- `add_walls`, `remove_walls`: TODO
- `run_update_volume`: Shrinks or expands the box to a given length.
- `run_langevin`: runs simulation with Langevin dynamics.
- `run_NPT`: runs simulation with the NPT ensemble.
- `run_NVT`: runs simulation with the NVT ensemble.
- `run_NVE`: runs simulation with the NVE ensemble.
- `run_displacement_cap`: runs NVE simulation with a cap on the maximum distance travelled by particles.
- `pickle_forcefield`: create a pickle file to save the force objects. This can be used to recreate a simulation class without the need to initiate `Molecule` and `System` classes.
- `save_restart_gsd`: saves a snapshot of the current state of the system. This can be used to recreate a simulation without the need to initiate `Molecule` and `System` classes.

### Resume a simulation

Resuming a simulation in flowerMD is easy, without the need to recreate the `Molecule` and `Simulation` classes. All that's required are two key components: a gsd snapshot of the system and a list of HOOMD force objects.

Users can save the snapshot of a simulation's latest state by invoking the `simulation.save_restart_gsd()` method. By default, this snapshot is saved in a file named `restart.gsd`. However, users have the flexibility to specify a different file path by configuring `file_path` parameter.

Additionaly, the list of HOOMD force objects can be preserved in a pickled format using `simulation.pickle_forcefield()` method. Pickled list is saved in `forcefield.pickle` by default. The file path can be configured with `file_path` parameter.
Please note that the user will need to pass the unpickled list to simulation class in order to recreate the class.

In [ ]:
from flowermd.base import Molecule, Pack, Simulation
from flowermd.library import OPLS_AA

molecule = Molecule(num_mols=30, smiles="c1ccccc1")

system = Pack(molecules=molecule, density=0.5)
system.apply_forcefield(r_cut=2.5, force_field=OPLS_AA(), auto_scale=True)

sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=system.hoomd_forcefield)

sim.run_NVE(n_steps=1e3)

In [ ]:
sim.save_restart_gsd("snapshot.gsd")

sim.pickle_forcefield("hoomd_forces.pickle")

In [ ]:
import pickle

f = open("hoomd_forces.pickle", "rb")
hoomd_ff = pickle.load(f)

In [ ]:
resume_sim = Simulation(initial_state="snapshot.gsd", forcefield=hoomd_ff)